## I. Setup, Installations, and Imports

#### (a) Installations

In [ ]:
! pip install spacy
! pip install nltk
! python -m spacy download en_core_web_sm
! pip install svgling

#### (b) Imports

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer

#### (c) Downloads

In [ ]:
nltk.download('words')
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('averaged_perceptron_tagger')
nltk.download('state_union')

## II. Named Entity Recognizer Function
Function to find the named entity recognizer of a sentence 

In [ ]:
def ner(sentence): 
    
    words = nltk.word_tokenize(sentence) #break down the sentence into words
    tagged = nltk.pos_tag(words) #tag the words with Part of Speech 
    chunks = nltk.ne_chunk(tagged, binary=False) #binary = False named entities are classified (i.e PERSON, ORGANIZATION)
    
    entities = []
    
    for chunk in chunks:
        if hasattr(chunk, 'label'): #hasattr(obj, key) -- checking if chunks have a label or not 
              entities.append(' '.join(c[0] for c in chunk)) #append entities to array
    
    return entities

## III. Filename Traversal

In [3]:
filename = 'inputs/ex10.txt'

In [ ]:
fp = open(filename, "r")
raw_text_file = fp.read() #read in the raw text file

In [9]:
my_dic = {}

In [7]:
myarray = [1,2,3,4]

for i in range(3):
    print(i)
    my_dic['result_' + str(i)] = i

my_dic

0
1
2


{'inputs/ex10.txt': 2, 'result_0': 0, 'result_1': 1, 'result_2': 2}

In [ ]:
raw_sentences = ['Python is an interpreted, high-level and general-purpose programming language', 
        'Apple acquired Zoom in China on Wednesday 6th, May 2020. This news has made Apple and Google stock jump by 5% on Dow Jones in the United States of America',     
        'Pythons design philosophy emphasizes code readability with on Feb 13 during the Revolutionary War']


for sentence in raw_sentences:
      
    #Get the entities of the sentence using the NER function above 
    entities = ner(sentence) 
    
    #Combine the sentence and entities into one array (get the final value for the dict)
    combined = [sentence, entities] 
    print(combined)
    
    #my_dic.update({filename: combined})
    my_dic[filename] = "test"

    
    #combined.clear() #reset combined array to allow for next sentence to parse
    
    


In [ ]:
my_dic

In [ ]:
raw_sentences = nltk.sent_tokenize(raw_text_file) #break text file into indiv sentences

#entities = []
#combined = []

for sentence in raw_sentences:
    
    print("1.________SENTENCE__________")
    print(sentence)
    
    
    print("2.________ENTITIES__________")
    entities = ner(sentence)
    print(entities)
    
    print("3.________LIST -- SENTENCE + ENTITIES__________")
    
    combined = [sentence, entities] #combine the sentence and entities into one array (get the final value for the dict)
    print(combined)
    combined.clear() #reset combined array to allow for next sentence to parse 
    
    print("")
    print("")
    